In [1]:
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
def __init__(self, layer_sizes, activation='relu', seed=42):
    np.random.seed(seed)
    self.layer_sizes = layer_sizes
    self.num_layers = len(layer_sizes) - 1
    self.activation = activation

    # Initialize weights and biases using Xavier initialisation
    self.weights = []
    self.biases = []

    for i in range(self.num_layers):
        weight_matrix = np.random.randn(layer_sizes[i], layer_sizes[i+1]) * np.sqrt(2 / (layer_sizes[i] + layer_sizes[i+1]))
        bias_vector = np.zeros((1, layer_sizes[i+1]))
        self.weights.append(weight_matrix)
        self.biases.append(bias_vector)
    
    print(f"Initialized network with layer sizes: {layer_sizes}")
    print(f"Activation function: {activation}")
    print(f"Number of layers: {self.num_layers}")

In [3]:
def count_parameters(self):
    total_params = 0
    for w, b in zip(self.weights, self.biases):
        total_params += w.size + b.size
    return total_params

Activation functiosn with their respective derivative

In [4]:
def relu(self, x):
    return np.maximum(0, x)
def relu_derivative(self, x):
    return (x > 0).astype(float)

def tanh(self, x):
    return np.tanh(x)
def tanh_derivative(self, x):
    return 1 - np.tanh(x)**2

def sigmoid(self, x):
    return 1 / (1 + np.exp(-x))
def sigmoid_derivative(self, x):
    sig = self.sigmoid(x)
    return sig * (1 - sig)

def softmax(self, x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def apply_activation(self, x):
    if self.activation == 'relu':
        return self.relu(x)
    elif self.activation == 'tanh':
        return self.tanh(x)
    elif self.activation == 'sigmoid':
        return self.sigmoid(x)
    else:
        raise ValueError("Unsupported activation function: {activation_type}")

Forward propagation throughout the network

mathematical formulation: 
    For each layer *l*: 
    Z[l] = A[l-1] @ W[l] + b[l] (Linear transformation)
    A[l] = activation(Z[l]) (Activation Function)

    Final Layer uses Softmax for classifcation

    Args: 
        X: Input data (batch_size, input_features)
        return_cache: If true return immediate values for back prop

    returns: 
        Predictions: Final output probaility (batch_size, num_classes)
        cache: Dictionary of intermediate values (if return_cache=True)

In [ ]:
def forward_pass(self, X, return_cache):
    cache = {}
    A = X
    cache['A0'] = X # Store input as activation of layer 0

    for i in range(self.num_layers):
        # Hidden Layers
        Z = np.dot(A, self.weights[i]) + self.biases[i]
        if i < self.num_layers - 1:
            A = self.apply_activation(Z)
        else:
            A = self.softmax(Z)  # Output layer with softmax
        
        cache[f'Z{i+1}'] = Z
        cache[f'A{i+1}'] = A

    if return_cache:
        return A, cache
    return A

def predict(self, X):
    probabilities = self.forward_pass(X, return_cache=False)
    return np.argmax(probabilities, axis=1)